In [1]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool
import os
import requests
import json
ETHERSCAN_KEY = os.getenv('ETHERSCAN_KEY')
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_KEY')
OX_KEY=os.getenv('OX_KEY')
llm = OpenAI(temperature=0)

In [2]:
@tool
def get_abi(address: str) -> str:
    """Gets the abi for the provided contract address"""
    response = requests.get("https://api.etherscan.io/api?module=contract&action=getabi&address="+address+"&apikey="+ETHERSCAN_KEY)
    abi = response.json()['result']
    abiJson = (json.loads(abi))
    return abiJson

get_abi

StructuredTool(name='get_abi', description='get_abi(address: str) -> str - Gets the abi for the provided contract address', args_schema=<class 'pydantic.main.get_abiSchemaSchema'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, func=<function get_abi at 0x12f4cc310>, coroutine=None)

In [3]:
@tool
def get_functions(abiJson: str) -> str:
    """Gets the abi for the provided contract address"""
    functions = []
    for item in abiJson:
        if item['type'] == 'function' and item['stateMutability'] != 'view':
            functions.append(item['name'])
    return functions

get_functions

StructuredTool(name='get_functions', description='get_functions(abiJson: str) -> str - Gets the abi for the provided contract address', args_schema=<class 'pydantic.main.get_functionsSchemaSchema'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, func=<function get_functions at 0x12f4cc3a0>, coroutine=None)

In [4]:
@tool
def encoded_transaction(inputs: str) -> str:
    """Encodes the transaction with the provided inputs"""
    

In [5]:
@tool
def get_parameters(dets:str) -> str:
    """Gets the function parameters for the given function from the abi"""
    address, functionName = dets.split(' ')
    abiJson = get_abi(address)
    for item in abiJson:
        if item['name'] == functionName and item['type'] == 'function' and item['stateMutability'] != 'view':
            return item['inputs']

In [13]:
@tool
def get_swap_rates(dets:str) -> str:
    """Gets the swap rates for the given token"""
    token1, token2 = dets.split('/')
    url = "https://api.0x.org/swap/v1/quote?sellToken="+token1+"&buyToken="+token2+"&sellAmount=1000000000000000000"
    response = requests.get(url)
    return response.json()['price']

get_swap_rates

StructuredTool(name='get_swap_rates', description='get_swap_rates(dets: str) -> str - Gets the swap rates for the given token', args_schema=<class 'pydantic.main.get_swap_ratesSchemaSchema'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, func=<function get_swap_rates at 0x1415bac10>, coroutine=None)

In [14]:
tools = [
    Tool.from_function(
        func=get_abi,
        name = "get_abi",
        description="gets abi"
        # coroutine= ... <- you can specify an async method if desired as well
    ),
    Tool.from_function(
        func=get_functions,
        name = "get_functions",
        description="gets functions from abi"
    )
    ,
    Tool.from_function(
        func=get_parameters,
        name = "get_parameters",
        description="Gets the function parameters for the given function from the abi"
    ),
    Tool.from_function(
        func=get_swap_rates,
        name = "get_swap_rates",
        description="Gets the swap rates for the given token"
    )
]
# tools = load_tools(["get_abi"], llm=llm)

agent = initialize_agent(tools,llm,AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [14]:
agent.run("get me abi for 0xdAC17F958D2ee523a2206206994597C13D831ec7")

"The abi for 0xdAC17F958D2ee523a2206206994597C13D831ec7 is [{'constant': True, 'inputs': [], 'name': 'name', 'outputs': [{'name': '', 'type': 'string'}], 'payable': False, 'stateMutability': 'view', 'type': 'function'}, {'constant': False, 'inputs': [{'name': '_upgradedAddress', 'type': 'address'}], 'name': 'deprecate', 'outputs': [], 'payable': False, 'stateMutability': 'nonpayable', 'type': 'function'}, {'constant': False, 'inputs': [{'name': '_spender', 'type': 'address'}, {'name': '_value', 'type': 'uint256'}], 'name': 'approve', 'outputs': [], 'payable': False, 'stateMutability': 'nonpayable', 'type': 'function'}, {'constant':"

In [19]:
agent.run("what functions are possible on 0xdAC17F958D2ee523a2206206994597C13D831ec7")

'The functions possible on 0xdAC17F958D2ee523a2206206994597C13D831ec7 are: name, deprecate, approve, deprecated, addBlackList, totalSupply, transferFrom, upgradedAddress, balances, decimals, maximumFee, _totalSupply, unpause, getBlackListStatus, allowed, paused, balanceOf, pause, getOwner, owner, symbol, transfer, setParams, issue, redeem, allowance, basisPointsRate, isBlackListed, removeBlackList, MAX_UINT, transferOwnership, destroyBlackFunds, and constructor.'

In [30]:
agent.run("what are the parameters for the function approve on 0xdAC17F958D2ee523a2206206994597C13D831ec7")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "get_parameters",
  "action_input": "0xdAC17F958D2ee523a2206206994597C13D831ec7 approve"
}
```


Observation: [{'name': '_spender', 'type': 'address'}, {'name': '_value', 'type': 'uint256'}]
Thought: I have the parameters for the function
Action:
```
{
  "action": "Final Answer",
  "action_input": "The parameters for the function approve on 0xdAC17F958D2ee523a2206206994597C13D831ec7 are _spender (address) and _value (uint256)."
}
```


> Finished chain.


'The parameters for the function approve on 0xdAC17F958D2ee523a2206206994597C13D831ec7 are _spender (address) and _value (uint256).'

In [ ]:
agent.run("Can you create an encoded transaction with the value of 0x0d1d4e623D10F9FBA5Db95830F7d3839406C6AF2 for spender & value of 100000000000000000")

In [15]:
agent.run("i want to swap usdt for eth")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "get_swap_rates",
  "action_input": "USDT/ETH"
}
```


Observation: 0.000000037758419728
Thought: I have the swap rate, now I need to provide the final answer
Action:
```
{
  "action": "Final Answer",
  "action_input": "The current swap rate for USDT to ETH is 0.000000037758419728"
}
```



> Finished chain.


'The current swap rate for USDT to ETH is 0.000000037758419728'